In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random 

import torch
import torch.nn as nn
from torch import optim 
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [28]:
# If any other format than ASCII then convert to ASCII 
def unicodeToASCII(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='Mn')
    
# lowercase and remove punctuation and non letter characters
def normalizeString(s):
    s = unicodeToASCII(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [29]:
# writing Regular Expressions?????

In [39]:
def readLangs(lang1, lang2, reverse=False):
    print ("reading lines...")
    
    # read file and split into lines
    lines = open('data/eng-fra.txt', encoding='utf-8').read().strip().split('\n')
    
    # split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in  l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [42]:
max_length = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length and p[1].startswith(eng_prefixes)
    #return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length 
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [43]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print ("Read sentence pairs: " + str(len(pairs)))
    
    pairs = filterPairs(pairs)
    print ("Length of trimmed sentence pairs: " + str(len(pairs)))
    
    print ("counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print ("Counted Words:")
    
    print (input_lang.name, input_lang.n_words)
    print (output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)
print (random.choice(pairs))

reading lines...
Read sentence pairs: 135842
Length of trimmed sentence pairs: 10853
counting words...
Counted Words:
hin 4489
eng 2925
[u'tu es pleine de talent .', u'you re very talented .']


In [44]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.embedding(input_size,hidden_size) # ????????
        self.gru = nn.GRU(hidden_size,hidden_size) # learn about GRU cell?????
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden 
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device=device)

In [45]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1,1, self.hidden_size, device=device)